In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from xgboost import XGBRegressor,XGBClassifier
from sklearn.metrics import log_loss
from sklearn.svm import LinearSVC,LinearSVR,SVC,SVR
from scipy.stats import norm,skew

# Setting the number of columns to display
pd.set_option('display.max_columns', None)
# year
yr=2014

### detailed is from 2010 to 2018. but i am using from this 2010 to 2013

these are the full forms 

WFGM - field goals made (by the winning team)

WFGA - field goals attempted (by the winning team)

WFGM3 - three pointers made (by the winning team)

WFGA3 - three pointers attempted (by the winning team)

WFTM - free throws made (by the winning team)

WFTA - free throws attempted (by the winning team)

WOR - offensive rebounds (pulled by the winning team)

WDR - defensive rebounds (pulled by the winning team)

WAst - assists (by the winning team)

WTO - turnovers committed (by the winning team)

WStl - steals (accomplished by the winning team)

WBlk - blocks (accomplished by the winning team)

WPF - personal fouls committed (by the winning team)

In [2]:
cities=pd.read_csv("WPrelimData2018/WCities_PrelimData2018.csv")
game_cities=pd.read_csv("WPrelimData2018/WGameCities_PrelimData2018.csv")

tourney_seeds=pd.read_csv("WDataFiles/WNCAATourneySeeds.csv")

regular_season_results=pd.read_csv("WPrelimData2018/WRegularSeasonCompactResults_PrelimData2018.csv")
detailed_regular_season=pd.read_csv("WPrelimData2018/WRegularSeasonDetailedResults_PrelimData2018.csv")

tourney_results=pd.read_csv("WPrelimData2018/WNCAATourneyCompactResults_PrelimData2018.csv")
detailed_tourney=pd.read_csv("WPrelimData2018/WNCAATourneyDetailedResults_PrelimData2018.csv")


# regular_season_results=regular_season_results.loc[regular_season_results['Season']<=yr]
# tourney_results=tourney_results.loc[tourney_results['Season']<yr]

detailed_regular_season=detailed_regular_season.loc[detailed_regular_season['Season']<=yr]
detailed_tourney=detailed_tourney.loc[detailed_tourney['Season']<yr]

sample_submission=pd.read_csv("WDataFiles/WSampleSubmissionStage1.csv")

print("Regular season results",regular_season_results.shape,detailed_regular_season.shape)
print("Tourney results",tourney_results.shape,detailed_tourney.shape)

Regular season results (101450, 8) (25525, 34)
Tourney results (1260, 8) (252, 34)


In [3]:
detailed_tourney['Type']=1          # tournament
detailed_regular_season['Type']=0      #regular season
detailed_tourney=pd.concat([detailed_tourney,detailed_regular_season],axis=0,ignore_index=True)
print(detailed_tourney.shape)
detailed_tourney.head()

(25777, 35)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Type
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18,1
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22,1
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20,1
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24,1
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12,1


In [4]:
# adding seeds to tournnament results
def extract_seedint(seed):
    k=int(seed[1:])
    return k;
def region(seed):
    return seed[0];
tourney_seeds['seed_int']=tourney_seeds['Seed'].apply(extract_seedint)
tourney_seeds['region']=tourney_seeds['Seed'].apply(region)
tourney_seeds.drop(columns=['Seed'],inplace=True)

winseeds = tourney_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed','region':'WRegion'})
lossseeds = tourney_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed','region':'LRegion'})
df_dummy = pd.merge(left=tourney_results, right=winseeds, how='left', on=['Season', 'WTeamID'])
tourney_results= pd.merge(left=df_dummy, right=lossseeds,how='left',on=['Season', 'LTeamID'])

In [5]:
# the formulae for the possession in the paper is wrong..!!!
detailed_tourney['WPossessions']=detailed_tourney['WFGA']-detailed_tourney['WOR']+detailed_tourney['WTO']+0.475*detailed_tourney['WFTA']
detailed_tourney['LPossessions']=detailed_tourney['LFGA']-detailed_tourney['LOR']+detailed_tourney['LTO']+0.475*detailed_tourney['LFTA']

# in a game the possesion of the two teams are nearly same(+2 or -2). So we can take the average
# the details is given in the link  https://kenpom.com/blog/stats-explained/
detailed_tourney['Possessions']=detailed_tourney['WPossessions']*0.5+detailed_tourney['LPossessions']*0.5

# offensive and defensive effiencies by dividing the effincies with the average number
detailed_tourney['WOE']=(detailed_tourney['WScore']*100)/detailed_tourney['Possessions']
detailed_tourney['WDE']=(detailed_tourney['LScore']*100)/detailed_tourney['Possessions']

detailed_tourney['LOE']=(detailed_tourney['LScore']*100)/detailed_tourney['Possessions']
detailed_tourney['LDE']=(detailed_tourney['WScore']*100)/detailed_tourney['Possessions']

# adding seeds 
df_dummy = pd.merge(left=detailed_tourney, right=winseeds, how='left', on=['Season', 'WTeamID'])
detailed_tourney= pd.merge(left=df_dummy, right=lossseeds,how='left',on=['Season', 'LTeamID'])

detailed_tourney['WSeed'].fillna(0,inplace=True)
detailed_tourney['LSeed'].fillna(0,inplace=True)

# finding the seed difference
detailed_tourney['SeedDiff']=detailed_tourney['WSeed']-detailed_tourney['LSeed']

# dropping the region columns 
detailed_tourney.drop(columns=['WRegion','LRegion'],inplace=True)




print(detailed_tourney.shape)
detailed_tourney.head()

(25777, 45)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Type,WPossessions,LPossessions,Possessions,WOE,WDE,LOE,LDE,WSeed,LSeed,SeedDiff
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18,1,65.025,64.375,64.7000,106.646059,85.007728,85.007728,106.646059,4.0,13.0,-9.0
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22,1,71.025,70.125,70.5750,94.934467,93.517535,93.517535,94.934467,8.0,9.0,-1.0
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20,1,65.450,65.800,65.6250,109.714286,56.380952,56.380952,109.714286,2.0,15.0,-13.0
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24,1,72.350,72.450,72.4000,103.591160,84.254144,84.254144,103.591160,3.0,14.0,-11.0
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12,1,59.800,62.075,60.9375,101.743590,68.923077,68.923077,101.743590,5.0,12.0,-7.0


I will calculate the national offensive and defensive effi

May I will use this to find the adjusted offensive and defensive effi. But currently I dont know any formuale for calculating the adjusted effi

In [6]:
# now i have to calculate the adjusted OE,DE for winning team and losing team
# before that I have to calculate the team offensive and defensive efficiency overall games 
woe=pd.DataFrame()
wde=pd.DataFrame()
woe['TeamID']=detailed_tourney.groupby('WTeamID').WOE.sum().index
woe['wOE']=detailed_tourney.groupby('WTeamID').WOE.sum().values

wde['TeamID']=detailed_tourney.groupby('WTeamID').WDE.sum().index
wde['wDE']=detailed_tourney.groupby('WTeamID').WDE.sum().values

loe=pd.DataFrame()
lde=pd.DataFrame()
loe['TeamID']=detailed_tourney.groupby('LTeamID').LOE.sum().index
loe['lOE']=detailed_tourney.groupby('LTeamID').LOE.sum().values

lde['TeamID']=detailed_tourney.groupby('LTeamID').LDE.sum().index
lde['lDE']=detailed_tourney.groupby('LTeamID').LDE.sum().values

oe=pd.merge(left=woe,right=loe,how='outer',on=['TeamID'])
de=pd.merge(left=wde,right=lde,how='outer',on=['TeamID'])

oe.fillna(0,inplace=True)
de.fillna(0,inplace=True)

oe['OE']=oe['wOE']+oe['lOE']
oe.drop(columns=['wOE','lOE'],inplace=True)

de['DE']=de['wDE']+de['lDE']
de.drop(columns=['wDE','lDE'],inplace=True)

# number of games 
wgames=pd.DataFrame()
wgames['TeamID']=detailed_tourney['WTeamID'].value_counts().index
wgames['wcount']=detailed_tourney['WTeamID'].value_counts().values

lgames=pd.DataFrame()
lgames['TeamID']=detailed_tourney['LTeamID'].value_counts().index
lgames['lcount']=detailed_tourney['LTeamID'].value_counts().values


games=pd.merge(left=wgames,right=lgames,how='outer',on=['TeamID'])
games['wcount'].fillna(0,inplace=True)
games['lcount'].fillna(0,inplace=True)
games['number']=games['wcount']+games['lcount']
games.drop(columns=['wcount','lcount'],inplace=True)

oe=pd.merge(left=oe,right=games,how='outer',on=['TeamID'])
de=pd.merge(left=de,right=games,how='outer',on=['TeamID'])

oe['OE']=oe['OE']/oe['number']
de['DE']=de['DE']/de['number']

oe.drop(columns=['number'],inplace=True)
de.drop(columns=['number'],inplace=True)

print(oe.shape,de.shape)

national_oe_average=(oe['OE'].sum())/(oe['OE'].shape[0])
national_de_average=(de['DE'].sum())/(de['DE'].shape[0])
print("The national offensive average is",national_oe_average)
print("The national defensive average is",national_de_average)

(350, 2) (350, 2)
The national offensive average is 89.646629353
The national defensive average is 90.2045766602


## TRAIN DATA PREPROCESSING

In [7]:
def change(s):
    if s=='N':
        return 0;
    elif s=='H':
        return 1;
    else:
        return -1;

    

df_dummy=pd.merge(left=detailed_tourney
                  ,right=oe.rename(columns={'TeamID':'WTeamID','OE':'WOE_avg'}),
                  how='left',on=['WTeamID'])

detailed_tourney=pd.merge(left=df_dummy
                           ,right=de.rename(columns={'TeamID':'WTeamID','DE':'WDE_avg'})
                           ,how='left',on=['WTeamID'])

df_dummy=pd.merge(left=detailed_tourney,
                  right=oe.rename(columns={'TeamID':'LTeamID','OE':'LOE_avg'}),
                  how='left',on=['LTeamID'])

detailed_tourney=pd.merge(left=df_dummy,
                          right=de.rename(columns={'TeamID':'LTeamID','DE':'LDE_avg'}),
                          how='left',on=['LTeamID'])

# finding the effi margin
detailed_tourney['WEM']=detailed_tourney['WOE_avg']-detailed_tourney['WDE_avg']
detailed_tourney['LEM']=detailed_tourney['LOE_avg']-detailed_tourney['LDE_avg']

# adjusted effi/ This is what i understood from the definition https://kenpom.com/blog/ratings-glossary/
detailed_tourney['adj_WOE']=detailed_tourney['WOE_avg']*(national_oe_average/national_de_average)
detailed_tourney['adj_WDE']=detailed_tourney['WDE_avg']*(national_de_average/national_oe_average)

detailed_tourney['adj_LOE']=detailed_tourney['LOE_avg']*(national_oe_average/national_de_average)
detailed_tourney['adj_LDE']=detailed_tourney['LDE_avg']*(national_de_average/national_oe_average)


detailed_tourney=detailed_tourney.loc[:,['WLoc','WOE_avg','WDE_avg','LOE_avg','LDE_avg',
                                         'Type','SeedDiff','WEM','LEM','adj_WOE','adj_WDE','adj_LOE','adj_LDE']]
detailed_tourney['WLoc']=detailed_tourney['WLoc'].apply(change)
detailed_tourney['Result']=1

temp=pd.DataFrame()

# creating features
temp['WLoc']=-detailed_tourney['WLoc']

temp['WOE_avg']=detailed_tourney['LOE_avg']

temp['WDE_avg']=detailed_tourney['LDE_avg']

temp['LOE_avg']=detailed_tourney['WOE_avg']

temp['LDE_avg']=detailed_tourney['WDE_avg']

temp['Type']=detailed_tourney['Type']

temp['SeedDiff']=-detailed_tourney['SeedDiff']

temp['WEM']=detailed_tourney['LEM']

temp['LEM']=detailed_tourney['WEM']

temp['adj_WOE']=detailed_tourney['adj_LOE']

temp['adj_WDE']=detailed_tourney['adj_LDE']

temp['adj_LOE']=detailed_tourney['adj_WOE']

temp['adj_LDE']=detailed_tourney['adj_WDE']

temp['Result']=0

games=pd.concat([detailed_tourney,temp],axis=0,ignore_index=True)
# games=games.loc[games['Type']==1]
print(games.shape)
games.head()

(51554, 14)


,WLoc,WOE_avg,WDE_avg,LOE_avg,LDE_avg,Type,SeedDiff,WEM,LEM,adj_WOE,adj_WDE,adj_LOE,adj_LDE,Result
0,0,109.463474,77.730508,100.073478,87.087748,1,-9.0,31.732966,12.985731,108.786404,78.214291,99.454488,87.629769,1
1,0,98.677685,84.802500,92.637552,85.983735,1,-1.0,13.875185,6.653817,98.067328,85.330298,92.064556,86.518885,1
2,1,102.308244,78.706148,94.822348,75.221176,1,-13.0,23.602097,19.601172,101.675431,79.196003,94.235838,75.689341,1
3,1,97.471654,86.582319,91.275702,88.285375,1,-11.0,10.889336,2.990326,96.868757,87.121194,90.711129,88.834851,1
4,0,90.577332,85.708862,100.559213,82.354793,1,-7.0,4.868470,18.204420,90.017079,86.242301,99.937219,82.867357,1


## TEST PREPREOCESSING

In [8]:
def yr(s):
    yr,team1,team2=map(int,s.split('_'))
    return yr;

def t1(s):
    yr,team1,team2=map(int,s.split('_'))
    return team1;

def t2(s):
    yr,team1,team2=map(int,s.split('_'))
    return team2;

sample_submission['Season']=sample_submission['ID'].apply(yr)
sample_submission['WTeamID']=sample_submission['ID'].apply(t1)
sample_submission['LTeamID']=sample_submission['ID'].apply(t2)


df_dummy=pd.merge(left=sample_submission,
                  right=oe.rename(columns={'TeamID':'WTeamID','OE':'WOE_avg'}),
                  how='left',on=['WTeamID'])

sample_submission=pd.merge(left=df_dummy,
                           right=de.rename(columns={'TeamID':'WTeamID','DE':'WDE_avg'}),
                           how='left',on=['WTeamID'])

df_dummy=pd.merge(left=sample_submission,
                  right=oe.rename(columns={'TeamID':'LTeamID','OE':'LOE_avg'}),
                  how='left',on=['LTeamID'])

sample_submission=pd.merge(left=df_dummy,
                           right=de.rename(columns={'TeamID':'LTeamID','DE':'LDE_avg'}),
                         how='left',on=['LTeamID'])


# adding adjusted 
sample_submission['adj_WOE']=sample_submission['WOE_avg']*(national_oe_average/national_de_average)
sample_submission['adj_WDE']=sample_submission['WDE_avg']*(national_de_average/national_oe_average)

sample_submission['adj_LOE']=sample_submission['LOE_avg']*(national_oe_average/national_de_average)
sample_submission['adj_LDE']=sample_submission['LDE_avg']*(national_de_average/national_oe_average)


#adding effi margin
sample_submission['WEM']=sample_submission['WOE_avg']-sample_submission['WDE_avg']
sample_submission['LEM']=sample_submission['LOE_avg']-sample_submission['LDE_avg']


# adding seeds 
df_dummy = pd.merge(left=sample_submission, right=winseeds, how='left', on=['Season', 'WTeamID'])
sample_submission= pd.merge(left=df_dummy, right=lossseeds,how='left',on=['Season', 'LTeamID'])

sample_submission['WSeed'].fillna(0,inplace=True)
sample_submission['LSeed'].fillna(0,inplace=True)

sample_submission['SeedDiff']=sample_submission['WSeed']-sample_submission['LSeed']
sample_submission.drop(columns=['WRegion','LRegion'],inplace=True)

sample_submission['WLoc']=0
sample_submission['Type']=1

sample_submission.head()

,ID,Pred,Season,WTeamID,LTeamID,WOE_avg,WDE_avg,LOE_avg,LDE_avg,adj_WOE,adj_WDE,adj_LOE,adj_LDE,WEM,LEM,WSeed,LSeed,SeedDiff,WLoc,Type
0,2014_3103_3107,0.5,2014,3103,3107,97.473699,92.067014,91.181595,80.860244,96.87079,92.640025,90.617605,81.363506,5.406686,10.321351,13,15,-2,0,1
1,2014_3103_3113,0.5,2014,3103,3113,97.473699,92.067014,89.341823,85.714676,96.87079,92.640025,88.789212,86.248152,5.406686,3.627147,13,9,4,0,1
2,2014_3103_3119,0.5,2014,3103,3119,97.473699,92.067014,88.673982,85.715611,96.87079,92.640025,88.125502,86.249092,5.406686,2.958372,13,13,0,0,1
3,2014_3103_3124,0.5,2014,3103,3124,97.473699,92.067014,109.463474,77.730508,96.87079,92.640025,108.786404,78.214291,5.406686,31.732966,13,2,11,0,1
4,2014_3103_3140,0.5,2014,3103,3140,97.473699,92.067014,98.013056,84.765952,96.87079,92.640025,97.406810,85.293523,5.406686,13.247104,13,12,1,0,1


## TRAINING

In [9]:
# features taken
p=['WLoc','adj_WOE','adj_WDE','adj_LOE','adj_LDE']

X=games.loc[:,p].values
y=games.loc[:,'Result'].values
X_test=sample_submission.loc[:,p].values

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.1,shuffle=True,random_state=42)


print("Training Shape",X_train.shape)
print("Training Label Shape",y_train.shape)
print("Validation Shape",X_val.shape)
print("Validation Label Shape",y_val.shape)
print("Test Shape",X_test.shape)

Training Shape (46398, 5)
Training Label Shape (46398,)
Validation Shape (5156, 5)
Validation Label Shape (5156,)
Test Shape (8064, 5)


## LOGISTIC REGRESSION

In [10]:
param={'C':[0.001,0.003,0.006,0.01,0.02,0.03,0.04,0.05,0.06,1,3,6,10]}
lr=LogisticRegression()
gridsearch=GridSearchCV(lr,param_grid=param,scoring='neg_log_loss',cv=5)
gridsearch.fit(X_train,y_train)
print("The Best C is",gridsearch.best_params_['C'])

The Best C is 3


In [11]:
lr=LogisticRegression(C=gridsearch.best_params_['C'])
lr.fit(X_train,y_train)
print("The Training Log loss is",log_loss(y_train,lr.predict_proba(X_train)[:,1]))
print("The Validation Log loss is",log_loss(y_val,lr.predict_proba(X_val)[:,1]))

The Training Log loss is 0.537450178778
The Validation Log loss is 0.526844895901


In [12]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
print("The Training Log loss is",log_loss(y_train,xgb.predict_proba(X_train)[:,1]))
print("The Validation Log loss is",log_loss(y_val,xgb.predict_proba(X_val)[:,1]))

The Training Log loss is 0.529360108659
The Validation Log loss is 0.526979816097


In [13]:
sample_submission['Pred']=xgb.predict_proba(X_test)[:,1]
ans=sample_submission.loc[:,['ID','Pred']]
ans.head()

,ID,Pred
0,2014_3103_3107,0.533931
1,2014_3103_3113,0.597794
2,2014_3103_3119,0.624518
3,2014_3103_3124,0.072727
4,2014_3103_3140,0.394072


In [17]:
tourney_results.loc[tourney_results['Season']==2014].head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSeed,WRegion,LSeed,LRegion
1008,2014,138,3113,69,3435,61,N,0,9,Z,8,Z
1009,2014,138,3124,87,3443,74,H,0,2,Z,15,Z
1010,2014,138,3140,72,3301,57,N,0,12,W,5,W
1011,2014,138,3143,64,3200,63,N,0,7,Z,10,Z
1012,2014,138,3177,104,3328,100,N,0,7,W,10,W


In [18]:
ans.loc[ans['ID']=='2014_3103_3345']

,ID,Pred
44,2014_3103_3345,0.477892


In [16]:
ans.to_csv("dont_trust.csv",index=False)
print("Done")

Done
